In [1]:
import os
import subprocess
import io
import re

In [ ]:
with open('/includes/test.txt','w') as file:
    file.write('test')
    

In [7]:
#path='/home/jupyter/pols_analytics_general/pols_code/production_tables/polsbigquery/stage_actual_budget_expected/sql/stage_actual_budget_expected.sql'

#path='/home/jupyter/sqlx_maker/angelo_madness.sql'

#path='/home/jupyter/pols_analytics_general/pols_code/production_tables/edwa-global/pols_analytics/marketing_metrics_v2/sql/marketing_metrics_v2_parseable.sql'

path='path/query.sql'

file_path = re.search('(.*)/(.*)(.sql)',path).group(2)

new_path = os.path.join('../local_repo/',file_path)

In [8]:
os.chdir('current_directory')
if not os.path.exists(new_path):
    os.mkdir(new_path)
os.chdir(new_path)

In [9]:
def read_and_clean_file(path):
    f = open(path,'r').read()
    f = re.sub('(CREATE OR REPLACE)','$$$$$$\\1',f)
    f = re.sub('`','',f)
    
    return(f)

def create_constants_file(txt):
    
    #trim white spaces
    txt = re.sub('\s{2,}',' ',txt.replace('\n',''))
    #Remove SET DDL word
    txt = re.sub('SET ', '', re.search('SET.*',txt).group())
    
    #List Logic
    ## Combines list comprehensions to create js script
    if len(re.findall('=',txt))>1:
        txt_lst = [f"""const {re.search('(.*)(=)(.*).',x).group(1)} = "{re.search('(.*)(=)(.*).',x).group(3)}";\nmodule.exports = {{{re.search('(.*)(=)(.*).',x).group(1)}}};""" for x in txt.split(',')]
    else:
        txt_lst = [f"""const {re.search('(.*)(=)(.*).',txt).group(1)} = "{re.search('(.*)(=)(.*).',txt).group(3)}";\nmodule.exports = {{{re.search('(.*)(=)(.*).',txt).group(1)}}};"""]
        
    
        
    #returns list of constants
    if len(re.findall('=',txt))>1:
        const = [re.search('(.*)(=)(.*).',x).group(1).replace(' ','') for x in txt.split(',')]
    else:
        const = [re.search('(.*)(=)(.*).',txt).group(1).replace(' ','')]
        
    #writes file
    
    folder = 'includes'
    
    os.mkdir(folder)
    
    for i in range(0,len(const)):
        
        fpath = os.path.join(folder,f'{const[i]}.js')
        
        with open(fpath,'w') as file:
            file.write(txt_lst[i])
            
    return(const)

def fix_query_references(tables,queries):
    fixed_queries=[]
    for q in queries:
        for t in tables:
            if t in q:
                ref_name = '''${ref("'''  +   str(re.search(r'(.*)\.(.*)\.(.*)' , t).group(3))  +   '''")}'''              
                #print(ref_name)
                q = re.sub(t,ref_name,q,flags=re.MULTILINE)
        fixed_queries.append(q)
        
    return(fixed_queries)

def fix_query_constants(const,queries):
    fixed_queries=[]
    for q in queries:
        for c in const:
            if c in q:
                q = re.sub(c,"constants."+c,q,flags=re.MULTILINE)
        fixed_queries.append(q)
        
    return(fixed_queries)

def run_full_sqlx_parser(path):
    
    f = read_and_clean_file(path)
    
    try:
        txt = re.search('(SET)(.*);', f , re.IGNORECASE).group()
    
        const_list = create_constants_file(txt)
        
    except:
        
        print('No SET Statements')
        const_list=[]
    
    #clean_f = [x for x in f.split('$$$$$$') if not re.match('DECLARE',x)]
    clean_f = [x for x in [x for x in f.split('$$$$$$') if not re.match('DECLARE',x)] if x]
    
    tables = [re.search(r'(CREATE OR REPLACE TABLE)(.*)(AS)(.*)', x, flags=re.IGNORECASE).group(2).strip() for x in clean_f]
    
    print(tables)
    
    queries = [re.sub(r'CREATE OR REPLACE TABLE .* AS(.*)','\\1', x, flags=re.IGNORECASE) for x in clean_f]
    
    queries = fix_query_references(tables=tables,queries=queries)
    
    if const_list:
        queries = fix_query_constants(const=const_list,queries=queries)
        
    schemas = [re.search(r'(.*)\.(.*)\.(.*)' , x).group(2) for x in tables]
    table_names = [re.search(r'(.*)\.(.*)\.(.*)' , x).group(3) for x in tables]
    configs = [f"config {{type: 'table', schema: '{x}' }}" for x in schemas]
    
    file_text = []
    for i in range(0,len(tables)):
        file_text.append(configs[i]+'\n'+queries[i])
        
    folder = 'definitions'
    
    os.mkdir(folder)
        
    for i in range(0,len(tables)):
        
        fpath = os.path.join(folder,f'{table_names[i]}.sqlx')
        
        with open(fpath, 'w') as file:
            file.write(file_text[i])
            
    return(file_text)
    

In [ ]:
file_text = run_full_sqlx_parser(path)

In [435]:
f = read_and_clean_file(path)

In [436]:
txt = re.search('(SET)(.*);', f , re.IGNORECASE).group()
const_list = create_constants_file(txt)

In [437]:
tables = [re.search(r'^(CREATE OR REPLACE TABLE) (.*) (AS)(.*)', x).group(2) for x in clean_f]
clean_f = [x for x in f.split('$$$$$$') if not re.match('DECLARE',x)]
queries = [re.sub(r'^CREATE OR REPLACE TABLE .* AS(.*)','\\1', x) for x in clean_f]

In [438]:
queries = fix_query_references(tables,queries )

In [439]:
if const_list:
    queries = fix_query_constants(const=const_list,queries=queries)

In [440]:
schemas = [re.search(r'(.*)\.(.*)\.(.*)' , x).group(2) for x in tables]
table_names = [re.search(r'(.*)\.(.*)\.(.*)' , x).group(3) for x in tables]
configs = [f"config {{type: 'table', schema: '{x}' }}" for x in schemas]

In [441]:
file_text = []
for i in range(0,len(tables)):
    file_text.append(configs[i]+'\n'+fixed_queries[i])

In [447]:
for i in range(0,len(tables)):
    with open(f'{table_names[i]}.sqlx', 'w') as file:
        file.write(file_text[i])

In [ ]:
#Check test
re.findall('polsbigquery\.budget_exercise\.[A-Za-z0-9_]+',','.join(file_text))

In [ ]:
re.findall('\$\{.*\}',','.join(file_text))

In [571]:
#Check test
re.findall('constants.[A-Za-z_]+',','.join(file_text))

[]

In [400]:
#subprocess.getoutput('rm *.sqlx')
#subprocess.getoutput('rm *.js')

''